In [1]:
import os, sys, glob, pickle, configparser
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
exp_config = configparser.ConfigParser()
exp_config.read('../config')

# Set the directories
NSD_top_dir = exp_config['DIR']['NSD_top_dir']
FS_top_dir = os.path.join(NSD_top_dir, 'data', 'nsddata', 'freesurfer')
mask_output_dir =  os.path.join(NSD_top_dir, 
                                'intermediate', 
                                'masks', 
                                'HCP_MMP_ROIs')
sub_list = range(1, 9)

In [2]:
list_rois = [10, 11, 12] + list(range(67, 91)) + [96, 97, 98] +[170, 171]

In [3]:
for curr_sub in sub_list:
    native_dir = os.path.join(FS_top_dir,
                                   'subj0{}'.format(curr_sub))
    HCP_MMP_rois_label = pd.read_csv(os.path.join(native_dir,
                                             'label',
                                             'HCP_MMP1.mgz.ctab'),
                                sep=' ', skiprows=1,
                                 names=['index', 'label'])
    list_rois_names = [HCP_MMP_rois_label['label'].values[k-1] for k in list_rois]
    
    lh_HCP_MMP_dir = os.path.join(native_dir,
                         'label',
                         'lh.HCP_MMP1.mgz')
    lh_HCP_MMP = nib.load(lh_HCP_MMP_dir).get_fdata().squeeze()
    lh_mask = np.isin(lh_HCP_MMP, MPC_num_roi)
    
    rh_HCP_MMP_dir = os.path.join(native_dir,
                         'label',
                         'rh.HCP_MMP1.mgz')
    rh_HCP_MMP = nib.load(rh_HCP_MMP_dir).get_fdata().squeeze()
    rh_mask = np.isin(rh_HCP_MMP, MPC_num_roi)
    
    print(np.sum(lh_mask) + np.sum(rh_mask))
    
    
    np.save(os.path.join(mask_output_dir, 'lh.lPFC.sub0{}.npy'.format(curr_sub)), lh_mask)
    np.save(os.path.join(mask_output_dir, 'rh.lPFC.sub0{}.npy'.format(curr_sub)), rh_mask)

    # Save them ask freesurfer files
    mask = lh_mask.astype(int)
    ctab = np.array([[25, 25, 25, 0], [255, 255, 255, 255]])
    names = ['background', 'mask']

    nib.freesurfer.write_annot(os.path.join(mask_output_dir, 'lh.lPFC.sub0{}.annot'.format(curr_sub)), labels=mask, ctab=ctab, names=names)

    mask = rh_mask.astype(int)
    ctab = np.array([[25, 25, 25, 0], [255, 255, 255, 255]])
    names = ['background', 'mask']

    nib.freesurfer.write_annot(os.path.join(mask_output_dir, 'rh.lPFC.sub0{}.annot'.format(curr_sub)), labels=mask, ctab=ctab, names=names)